In [1]:
import os
for f in os.listdir():
    if f.startswith("Recipes") or f.endswith(".py") or f.endswith(".tokens"):
        os.remove(f)

In [2]:
%%writefile Recipes.g4
grammar Recipes;

program
    : statement+ EOF
    ;

statement
    : ingredientStmt
    | stepStmt
    | totalCostStmt
    | totalCaloriesStmt
    ;

ingredientStmt
    : 'ingredient' ID AMOUNT ('costPerUnit' FLOAT)? ('calories' INT)? ';'
    ;

stepStmt
    : 'step' ID stepParams? (stepModifiers | 'time' AMOUNT)* ';'
    ;

totalCaloriesStmt: 'totalCalories' ';';

stepParams : (ID | AMOUNT | FLOAT)+ ;

stepModifiers: (parallel | dependsOn)+;

parallel: 'parallel';

dependsOn: 'dependsOn' ID;

totalCostStmt: 'totalCost' ';';

ID      : [a-zA-Z_][a-zA-Z_0-9]* ;

AMOUNT
    : [0-9]+ ('g' | 'kg' | 'ml' | 'l' | 'min' | 'h' | 'C' | 'piece' | 'cup' | 'tbsp' | 'tsp')
    ;

INT : [0-9]+ ;

FLOAT   : [0-9]+ ('.' [0-9]+)? ;
WS      : [ \t\r\n]+ -> skip ;
COMMENT : '//' ~[\r\n]* -> skip ;

Writing Recipes.g4


In [3]:
!curl -O https://www.antlr.org/download/antlr-4.9.3-complete.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3425k  100 3425k    0     0  7400k      0 --:--:-- --:--:-- --:--:-- 7399k


In [4]:
!java -jar antlr-4.9.3-complete.jar -Dlanguage=Python3 -no-listener -visitor Recipes.g4

In [5]:
%%writefile EvalVisitor.py
from antlr4 import *
import re

if __name__ is not None and "." in __name__:
    from RecipesParser import RecipesParser
    from RecipesVisitor import RecipesVisitor
else:
    from RecipesParser import RecipesParser
    from RecipesVisitor import RecipesVisitor

class EvalVisitor(RecipesVisitor):
    def __init__(self):
        self.ingredients = {}
        self.total_cost = 0.0
        self.total_calories = 0
        self.steps = []

    def visitProgram(self, ctx):
        for st in ctx.statement():
            self.visit(st)
        return {
        "ingredients": self.ingredients,
        "steps": self.steps,
        "total_cost": self.total_cost,
        "total_calories": self.total_calories
        }

    def visitIngredientStmt(self, ctx):
      name   = ctx.ID().getText()
      amount = ctx.AMOUNT().getText()    # ejemplo: '200g'
      # separa numero y unidad con regex
      m = re.match(r'(\d+)([a-zA-Z]+)', amount)
      if m:
        qty  = int(m.group(1))         # 200
        unit = m.group(2)              # g
      else:
        qty, unit = 0, ''
      cost = float(ctx.FLOAT().getText()) if ctx.FLOAT() else 0.0
      calories = int(ctx.INT().getText()) if ctx.INT() else 0

      self.ingredients[name] = (qty, unit, cost)
      self.total_cost += cost
      self.total_calories += calories

    def visitStepStmt(self, ctx):
      step_name = ctx.ID().getText()

      # Parámetros (ingredientes o insumos del paso)
      params = [p.getText() for p in ctx.stepParams().children] if ctx.stepParams() else []

      # Tiempo
      time = None
      if ctx.AMOUNT():
          for a in ctx.AMOUNT():
              val = a.getText()
              if val.endswith("min") or val.endswith("h"):
                  time = val
                  break

      # Dependencias
      depends = []
      if ctx.stepModifiers():
          for mod in ctx.stepModifiers():
              if mod.dependsOn():
                  for dep in mod.dependsOn():
                      depends.append(dep.ID().getText())

      # Guardar
      step_info = {
          "step": step_name,
          "params": params,
          "time": time,
          "depends_on": depends
      }
      self.steps.append(step_info)

      # Imprimir más detallado
      msg = f"Paso: {step_name}"
      if params:
          msg += f" ({'mezclando ' + ', '.join(params) if step_name == 'mezclar' else 'usando ' + ', '.join(params)})"
      if time:
          msg += f" (tiempo: {time})"
      if depends:
          msg += f" (depende de: {', '.join(depends)})"
      print(msg)


    def visitTotalCostStmt(self, ctx):
      print("Costo total:", self.total_cost)

    def visitTotalCaloriesStmt(self, ctx):
      print("Calorías totales:", self.total_calories)


Writing EvalVisitor.py


In [7]:
from antlr4 import *
from RecipesLexer import RecipesLexer
from RecipesParser import RecipesParser
from EvalVisitor import EvalVisitor

input_text = """
ingredient harina 200g costPerUnit 0.01 calories 700;
ingredient leche 500ml costPerUnit 0.002 calories 250;
ingredient huevo 2piece costPerUnit 0.3 calories 150;
ingredient azucar 100g costPerUnit 0.005 calories 400;
ingredient mantequilla 50g costPerUnit 0.02 calories 360;

step mezclar harina leche huevo azucar time 10min;
step derretir mantequilla time 5min;
step combinar mezcla mantequilla time 3min dependsOn mezclar dependsOn derretir;
step hornear combinacion time 40min dependsOn combinar;

totalCost;
totalCalories;
"""

lexer = RecipesLexer(InputStream(input_text))
token_stream = CommonTokenStream(lexer)
parser = RecipesParser(token_stream)
tree = parser.program()

visitor = EvalVisitor()
visitor.visit(tree)


Paso: mezclar (mezclando harina, leche, huevo, azucar) (tiempo: 10min)
Paso: derretir (usando mantequilla) (tiempo: 5min)
Paso: combinar (usando mezcla, mantequilla) (tiempo: 3min) (depende de: mezclar, derretir)
Paso: hornear (usando combinacion) (tiempo: 40min) (depende de: combinar)
Costo total: 0.337
Calorías totales: 1860


{'ingredients': {'harina': (200, 'g', 0.01),
  'leche': (500, 'ml', 0.002),
  'huevo': (2, 'piece', 0.3),
  'azucar': (100, 'g', 0.005),
  'mantequilla': (50, 'g', 0.02)},
 'steps': [{'step': 'mezclar',
   'params': ['harina', 'leche', 'huevo', 'azucar'],
   'time': '10min',
   'depends_on': []},
  {'step': 'derretir',
   'params': ['mantequilla'],
   'time': '5min',
   'depends_on': []},
  {'step': 'combinar',
   'params': ['mezcla', 'mantequilla'],
   'time': '3min',
   'depends_on': ['mezclar', 'derretir']},
  {'step': 'hornear',
   'params': ['combinacion'],
   'time': '40min',
   'depends_on': ['combinar']}],
 'total_cost': 0.337,
 'total_calories': 1860}